In [ ]:
using LinearAlgebra
using JuMP
using Ipopt
using Plots

# Inicializar variables
function inicializar_variables(valores_iniciales, N)
    variables = zeros(N, 3)
    variables[1, :] = valores_iniciales
    return vec(variables')
end

# Kernel RBF
function kernel_rbf(x, x_gorro, l, i, sigma)
    return exp(-(1/(2*sigma^2)) * ((x[l, 1]-x_gorro[i, 1])^2 + (x[l, 2]-x_gorro[i, 2])^2))
end

# Función objetivo
function funcion_objetivo(vars, kernel, vf, lambda_value, N)
    x = reshape(vars[1:2*N], 2, N)'
    alpha = vars[2*N+1:end]
    return (x[N, 2]-vf)^2 + lambda_value*sum(alpha.^2)
end

# Restricción
function restriccion(vars, valores_iniciales, T, N, gamma, sigma)
    x = reshape(vars[1:2*N], 2, N)'
    alpha = vars[2*N+1:end]
    constraints = zeros(2*N)
    constraints[1:2] .= x[1, 1:2] .- valores_iniciales[1:2]
    ΔT = T / N
    for i in 1:N-1
        constraints[2*i+1] = x[i+1, 1] - x[i, 1] + ΔT * x[i, 2]
        constraints[2*i+2] = x[i+1, 2] - x[i, 2] + ΔT * (-gamma * x[i, 1] + x[i, 2] + sum(alpha[l] * kernel(x, x, l, i, sigma) for l in 1:N))
    end
    return constraints
end

# Graficar soluciones
function graficar_soluciones(result, valores_iniciales, T, N, vf, sigma)
    x_sol = reshape(result[1:2*N], 2, N)'
    alpha_sol = result[2*N+1:end]
    u_opt = [sum(alpha_sol[l] * kernel_rbf(x_sol, x_sol, l, i, sigma) for l in 1:N) for i in 1:N]

    println("Valor final de la función objetivo: ", (x_sol[N, 2] - vf)^2)
    println("Velocidad final: ", x_sol[N, 2])

    plotly()
    t = 0:T/N:T
    plot1 = plot(t, x_sol[:, 1], label="Posición x", xlabel="Tiempo", ylabel="Posición x", legend=:top, grid=true)
    plot2 = plot(t, x_sol[:, 2], label="Velocidad v", xlabel="Tiempo", ylabel="Velocidad v", legend=:top, grid=true, color=:orange)
    plot3 = plot(t, u_opt, label="Control Óptimo", xlabel="Tiempo", ylabel="Control", legend=:top, grid=true)

    plot(plot1, plot2, plot3, layout=(3, 1))
end

# Parámetros
T, N, vf, lambda_value, sigma, gamma = 10, 30, 100, 0.0001, 1, 1
valores_iniciales = [80, 80, 0.1]
variables_iniciales = inicializar_variables(valores_iniciales, N)

# Definir y resolver el problema de optimización con JuMP
model = Model(Ipopt.Optimizer)
@variable(model, vars[1:3*N])

# Definir la función objetivo
@NLobjective(model, Min, funcion_objetivo(vars, kernel_rbf, vf, lambda_value, N))

# Definir las restricciones
for i in 1:2*N
    @NLconstraint(model, restriccion(vars, valores_iniciales, T, N, gamma, sigma)[i] == 0)
end

# Resolver el problema de optimización
optimize!(model)
resultado = value.(vars)

# Ejecución de la optimización
println("Resultado de la optimización: ", resultado)

# Graficar las soluciones
graficar_soluciones(resultado, valores_iniciales, T, N, vf, sigma)
